In [1]:
import numpy as np
import datasets
from rich.console import Console
from rich.table import Table
from rich.layout import Layout

In [2]:
dataset = datasets.Dataset.from_parquet("embeddings.parquet")

# embeddings must be numpy arrays
dataset.set_format(type='numpy', columns=['embedding'], output_all_columns=True)
dataset

Using custom data configuration default-142e43c20aee2ea2
/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.9/site-packages/datasets/builder.py:712: FutureWarning: 'use_auth_token' was deprecated in version 2.7.1 and will be removed in 3.0.0. Pass `use_auth_token` to the initializer/`load_dataset_builder` instead.
  warnings.warn(
Found cached dataset parquet (/Users/maxwoolf/.cache/huggingface/datasets/parquet/default-142e43c20aee2ea2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['id', 'name', 'num_tokens', 'embedding'],
    num_rows: 1154
})

Because the embeddings are already unit normalized, the cosine similarity can be calculated just by calculating the dot product, which in Python is easy since you can do `A @ B`!

In [3]:
# Pikachu and Raichu
dataset[24]["embedding"] @ dataset[25]["embedding"]

0.961932

However, because the input data is semantically similar, cosine similarity will always be very high (in testing, the lowest is `0.88`, so rescaling is an option)

Let's try inputting your own ID. Feel free to select your own Pokemon as the target ID to find similarity results you want!

In [4]:
target_id = 1

# instead of looking at array indices, we need to find an exact match in case for certain IDs
target_poke = dataset.filter(lambda x: x["id"] == target_id)[0]
target_poke["name"]

Loading cached processed dataset at /Users/maxwoolf/.cache/huggingface/datasets/parquet/default-142e43c20aee2ea2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-1db0bacecf6d748b.arrow


'bulbasaur'

However, to compare one Pokemon with all Pokemon, it's much faster to do the dot product of all Pokemon embeddings at the same time, then select the top ones (ignoring the top match, which will always be the original one)

In [5]:
cos_sims = np.dot(dataset["embedding"], target_poke["embedding"])

# min-max norm to scale to 0-1: comment out if do not want to rescale.
cos_sims = (cos_sims - cos_sims.min()) / (cos_sims.max() - cos_sims.min())

# argsort sorts ascending, so must reverse order to get most similar.
top_indicies = np.argsort(cos_sims)[::-1]
top_indicies

array([   0,    1,    2, ..., 1130,  692,  894])

Map these indicies to the dataset.

For fun, we'll style the similarity results as a table using `rich`.

In [6]:
top_n = 25
filtered_indices_pos = top_indicies[1 : (top_n + 1)]
filtered_indices_neg = top_indicies[-(top_n):][::-1]

layout = Layout()
layout.split_row(
    Layout(name="left"),
    Layout(name="right"),
)

# Top N Pokemon
pos_table = Table(title=f"Pokémon Most Similar to {target_poke['name']}")

pos_table.add_column("ID", style="bold")
pos_table.add_column("Pokémon")
pos_table.add_column("Similarity", style="green")

for index in filtered_indices_pos:
    idx = int(index)
    pos_table.add_row(str(dataset[idx]["id"]), dataset[idx]["name"], f"{cos_sims[idx]:.3%}")

layout["left"].update(pos_table)

# Bottom N Pokemon
neg_table = Table(title=f"Pokémon Least Similar to {target_poke['name']}")

neg_table.add_column("ID", style="bold")
neg_table.add_column("Pokémon")
neg_table.add_column("Similarity", style="cyan")

for index in filtered_indices_neg:
    idx = int(index)
    neg_table.add_row(str(dataset[idx]["id"]), dataset[idx]["name"], f"{cos_sims[idx]:.3%}")

layout["right"].update(neg_table)

console = Console(width=90, height=top_n+10)
console.print(layout)

   Pokémon Most Similar to bulbasaur              Pokémon Least Similar to bulbasaur      
┏━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓     ┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ ID    ┃ Pokémon         ┃ Similarity ┃     ┃ ID    ┃ Pokémon              ┃ Similarity ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩     ┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ 2     │ ivysaur         │ 67.147%    │     │ 895   │ regidrago            │ 0.000%     │
│ 3     │ venusaur        │ 61.266%    │     │ 693   │ clawitzer            │ 2.852%     │
│ 406   │ budew           │ 57.669%    │     │ 10226 │ urshifu-single-stri… │ 3.116%     │
│ 10033 │ venusaur-mega   │ 56.869%    │     │ 10207 │ garbodor-gmax        │ 5.100%     │
│ 69    │ bellsprout      │ 56.698%    │     │ 575   │ gothorita            │ 5.350%     │
│ 46    │ paras           │ 55.139%    │     │ 467   │ magmortar            │ 5.548%     │
│ 387   │ turtwig         │ 52.325%    │     │ 10227 │ urshifu-rapid-strik… │ 5.817%     │
│ 393   │ piplup          │ 51.747%    │     │ 473   │ mamoswine            │ 6.290%     │
│ 4     │ charmander      │ 50.761%    │     │ 10215 │ coalossal-gmax       │ 6.820%     │
│ 495   │ snivy           │ 50.669%    │     │ 10126 │ lycanroc-midnight    │ 7.165%     │
│ 155   │ cyndaquil       │ 50.641%    │     │ 10242 │ goodra-hisui         │ 7.636%     │
│ 432   │ purugly         │ 50.328%    │     │ 615   │ cryogonal            │ 8.163%     │
│ 47    │ parasect        │ 50.271%    │     │ 10151 │ rockruff-own-tempo   │ 8.203%     │
│ 511   │ pansage         │ 48.572%    │     │ 10225 │ duraludon-gmax       │ 8.294%     │
│ 7     │ squirtle        │ 48.526%    │     │ 10152 │ lycanroc-dusk        │ 8.298%     │
│ 231   │ phanpy          │ 48.472%    │     │ 807   │ zeraora              │ 8.671%     │
│ 585   │ deerling        │ 48.082%    │     │ 896   │ glastrier            │ 8.701%     │
│ 37    │ vulpix          │ 47.960%    │     │ 144   │ articuno             │ 8.750%     │
│ 220   │ swinub          │ 47.604%    │     │ 176   │ togetic              │ 9.116%     │
│ 74    │ geodude         │ 47.549%    │     │ 10239 │ zoroark-hisui        │ 10.189%    │
│ 580   │ ducklett        │ 47.529%    │     │ 62    │ poliwrath            │ 10.418%    │
│ 422   │ shellos         │ 47.518%    │     │ 159   │ croconaw             │ 10.460%    │
│ 10166 │ farfetchd-galar │ 47.360%    │     │ 10236 │ samurott-hisui       │ 10.735%    │
│ 83    │ farfetchd       │ 46.839%    │     │ 226   │ mantine              │ 10.890%    │
│ 762   │ steenee         │ 46.827%    │     │ 126   │ magmar               │ 10.916%    │
└───────┴─────────────────┴────────────┘     └───────┴──────────────────────┴────────────┘